In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import time
import datetime
import io
import sqlite3
import pyodbc 
import threading

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(chrome_options=options)

<ipython-input-2-b5f1867c00b1>:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


In [3]:
driver.get("https://www.google.com")

In [4]:
button=driver.find_element_by_xpath("//input")

In [5]:
button.send_keys("google play pinterest")

In [6]:
button.send_keys(Keys.RETURN)

In [7]:
#serach for the applcation in the Google play
try:
    link=WebDriverWait(driver,10).until(
        expected_conditions.presence_of_all_elements_located(
            (By.PARTIAL_LINK_TEXT, "Pinterest – Apps on Google Play")
        )                
    )
    link[0].click()
except Exception as e:
    print(e)

In [8]:
# def create_table_pintrest():
#     conn = sqlite3.connect(database='hesam.db')
#     cur = conn.cursor()
#     cur.execute(
#         # 'comment', 'author', 'rate', 'cr_date','likes'
#         'CREATE TABLE IF NOT EXISTS [pintrest]  (comment text,author text, rate int,cr_date text, likes int  )')
#     conn.commit()
#     conn.close()
# # insert

# def insert(comment ,author , rate ,cr_date , likes):
#     conn = sqlite3.connect(database='hesam.db')
#     cur = conn.cursor()
#     cur.execute("insert into pintrest values(?,?,?,?,?)", (comment ,author , rate ,cr_date , likes ))
#     conn.commit()
#     conn.close()
    
# def select():
#     conn = sqlite3.connect('hesam.db')
#     cur = conn.cursor()
#     cur.execute('select * from pintrest')
#     t1 = cur.fetchall()
#     conn.close()
#     return t1

# def delete():
#     conn = sqlite3.connect('hesam.db')
#     cur = conn.cursor()
#     cur.execute("delete from pintrest ")
#     conn.commit()
#     conn.close()
class Multi_ins(threading.Thread):
    def insert(self,comment, author, rating,date, likes):
        connection = pyodbc.connect('Driver={SQL Server};'
                                    'Server='+'DESKTOP-RT88KQU'+';'
                                    'Database='+'socialMedia'+';'
                                    'user=sa;'
                                    'password=21')
        cursor = connection.cursor()
        cursor.execute("""
        INSERT INTO [dbo].[pintrest]([comment],[author],[rating],[date],[likes])VALUES (?,?,?,?,?)""",
        comment, author, rating, date, likes)
        connection.commit()

In [9]:
#create_table_pintrest()

In [10]:
def ful_reviws(i):
    result ='ful_reviws'
    try:
        
        ful_reviws=driver.find_element_by_xpath(
            F'//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[2]/span[1]/div/button'
            )
        ful_reviws.click()

    except :
        pass

def get_comments(i):
    result='get_comments'
     
    try:
        result= driver.find_elements_by_xpath(
                F'//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[2]/span'
                )
        result=''.join ([x.text for x in result ])
        
    except Exception as e:
        print(e) 
    return result

def get_author(i):
    result='get_author'
    try:
        result= driver.find_element_by_xpath(
            f'//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[1]/div[1]/span'
        ).text
    except Exception as e:
        print(e) 
    return result 

def get_rating(i):
    result='get_rating'
    try:
        rating=driver.find_element_by_xpath(
            f'//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[1]/div[1]/div/span[1]/div/div'
        ).get_attribute("aria-label")
        result =rating [6:7]
    except Exception as e:
        print(e) 
    return result 

def get_date(i):
    result='get_date'
    try:
        date=driver.find_element_by_xpath(
            f'//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[1]/div[1]/div/span[2]'
        ).text
        result=  datetime.datetime.strptime( date.replace(',',''), '%B %d %Y')
    except Exception as e:
        print(e) 
    return result 

def get_like(i):
    result='get_like'
    try:
        result= driver.find_element_by_xpath(
            f'//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/div/div[1]/div[2]/div[1]/div[{i}]/div/div[2]/div[1]/div[2]/div/div[1]/div[2]'
        ).text
    except Exception as e:
        print(e) 
    return result 
             

In [11]:
#show complete comments in the pfirst page 
#Todo : compete this part for the future use in the loop part 

def get_info(i):
    #try to pen comments
    ful_reviws(i)
    #get comments
    comment=get_comments(i)

    author=get_author(i)
    rating=get_rating(i)
    date=get_date(i)
    likes=get_like(i)

    # print(f"author:{author} rate: {rating} cr_date: {date} likes: {likes}")
    if (comment and comment !='get_comments' and comment !='') :
        Multi_ins().insert(comment,author,rating ,date , likes  )




In [12]:
#click on the read all revies for the first time in the page
try:
    read_all_reviews=WebDriverWait(driver, 4).until(
        expected_conditions.presence_of_element_located((By.XPATH, '//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[6]/div/span/span'))
    )  
except Exception as e:
    print(e)
read_all_reviews.click()
#//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[6]/div/span/span

In [13]:
j=1
flag=True
body = driver.find_element_by_css_selector('body')

while flag:
    # if j >4600:
    
    get_info(j)

    j+=1
    if j % 40 == 0 :
        for zi in range(min ((j//50)+1, 2)):
            #driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            try:
                showmore=WebDriverWait(driver, 0.3).until(
                    expected_conditions.presence_of_element_located(
                        (By.XPATH,'//div[@class="PFAhAf"]/div[@role="button"]/span[@class="CwaK9"]/span[@class="RveJvd snByac"]')
                    )
                )
                showmore.click()
            except Exception as  e:
                pass
    if j % 50 == 0 :
        print(j)
    
                
  

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
Message: disconnect

KeyboardInterrupt: 